# Advection Equation 

## FTCS method

Here we solve the advection equation $u_t + a u_x=0$ using as an initial condition a gaussian profile $u(x,t=0)=\exp{[-(x-x_0)^2]}$ with $x_0=5$.

We use a domain $x\in [0,10]$ with periodic boundary conditions.

In [1]:
import numpy as np

#define the speed a
a=1.0

# Define the domain
L = 10.0     # Domain length
nx = 101    # Number of grid points
dx = L/(nx-1)   # Grid spacing
x = np.linspace(0, L, nx)

##print(x,'\n',dx) #just a check

# Define the time step and the final time
cf = 0.5
dt = cf*dx/a   # Time step
t_final = 20.0

print('nx=',nx)
print('dx=',dx)
print('dt=',dt)
print('Number of iterations=',t_final/dt)


# Define the initial condition
x0=5
u_initial = np.exp(-(x-x0)**2)

# Initialize the solution array
u_current = u_initial.copy()

#we create arrays where we will store the time and the l2norm
l2norm=[]
l2norm.append(np.sqrt(np.sum(u_current**2)/len(u_current)))

time=[]
time.append(0.0)

nx= 101
dx= 0.1
dt= 0.05
Number of iterations= 400.0


Here we set the directory for our first animation: 

In [2]:
import matplotlib.pyplot as plt
import os

##create directory where to save images
print(os.getcwd())

os.makedirs('./images/dx_20', exist_ok=True)

os.listdir('./')

/Users/nicolegrillo/Desktop/Numerical Relativity/numerical_relativity_2023/Advection


['l2norm_FTCS.dat',
 'Advection_Equation_LW.ipynb',
 '.DS_Store',
 'images',
 'Advection_Equation_Leapfrog.ipynb',
 'u_Leapfrog.gif',
 'u_LAX.gif',
 '1case_animation.gif',
 'README.md',
 'u_LW.gif',
 'l2norm_LAXWENDROFF.dat',
 '.ipynb_checkpoints',
 'u_FTCS.gif',
 'l2norm_LAX.dat',
 'l2norm_LEAPFROG.dat',
 'Advection_Equation_LF.ipynb',
 'Advection_Equation_FTCS.ipynb',
 'movie.mp4']

## Solution of the equation: 

In [ ]:
# Initilize time and iteration counter
t = 0.0
i = 0

#save the initial conditions
plt.plot(x, u_current)
plt.title('Time='+str(round(t,2)))
plt.ylim(0,1.1)
plt.savefig('./images/fig_'+str(i).zfill(5)+'.png', dpi=200)
plt.close()

total_sol = []

#solve the advection equation
while t < t_final:
    
    total_sol.append(u_current)
    # Compute the new solution using the FTCS method
    # Note: np.roll(u_current, -1) is equivalent to u(j+1) and
    #       np.roll(u_current,  1) is equivalent to u(j-1)
    # using np.roll is equivalent to use periodic boundary conditions
    u_next = u_current - a*dt/(2*dx)*(np.roll(u_current, -1) - np.roll(u_current, 1))    
    
    # Update the solution
    u_current = u_next.copy()
    
    
    #advance the time 
    t += dt
    i += 1
    
    #compute the l2 norm and add the time to the time vector
    l2norm.append(np.sqrt(np.sum(u_current**2)/len(u_current)))
    time.append(t)
    
    #plot the current result and save in an image every 10 iterations
    if (i%10==0):
        plt.plot(x, u_current)
        plt.title('Time='+str(round(t,2)))
        plt.ylim(0,1.1)
        plt.savefig('./images/fig_'+str(i).zfill(5)+'.png', dpi=200)
        plt.close()

In [ ]:
# Plot the final solution
plt.plot(x, u_initial, label='Initial')
plt.plot(x, u_current, label='Final')
#plt.title('Time='+str(round(t,2)))
plt.ylim(0,1.1)
plt.legend()
plt.title("Initial and final solution comparison")
plt.show()

Move on to produce the animation: 

In [ ]:
# set the directory where your images are stored
directory = "./images/dx_20"

# get the list of image files in the directory
files = os.listdir(directory)

# sort the files in alphanumeric order
files=sorted(files)

In [ ]:
import imageio
with imageio.get_writer('./movie.mp4', mode='I') as writer:  # , quality=10
    for file in files:
        image = imageio.imread('./images/dx_20/'+file)
        writer.append_data(image)
        

files=[]


In [ ]:
# don't worry about the code in this cell, it is just to let you 
# display the movies you generated above in Jupyter notebook
from IPython.display import HTML

HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="./movie.mp4" type="video/mp4">
</video></div>""")

See the values for the norm as time passes: 

In [ ]:
plt.plot(time,l2norm)
plt.show()

Compare the result with the stored data: 

In [ ]:
#Compare with data stored
l2_saved=np.genfromtxt('./l2norm_FTCS.dat', comments='#')
plt.plot(time,l2norm)
plt.plot(l2_saved[:,0], l2_saved[:,1], '+')
plt.show()

### FuncAnimation animations:

Here I just report an additional code I developed to see animation without having to save the images every time.

In [ ]:
import matplotlib.animation as animation

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel('x')
ax.set_ylabel('P')
plotLine, = ax.plot(x, np.zeros(len(x))*np.NaN, 'r-')
plotTitle = ax.set_title("t=0")
ax.set_ylim(0,1.)
ax.set_xlim(0,10)


def solution(t):
    p = np.array(total_sol)[t]
    return p


def animate(t):
    pp = solution(t)
    plotLine.set_ydata(pp)
    plotTitle.set_text(f"t = {t:.1f}")
    #ax.relim() # use if autoscale desired
    #ax.autoscale()
    return [plotLine,plotTitle]



ani = animation.FuncAnimation(fig, func=animate, frames=np.arange(0, len(total_sol), 5), blit=True)
ani.save('u_FTCS.gif')
plt.show()